In [9]:
from load_data import read_data, read_env
import os

ENV_PATH = os.getcwd()[:-3]
DATA_PATH = os.getcwd()[:-3] + "/data"

env_vars = read_env(ENV_PATH)
df = read_data(DATA_PATH)

GPT_API_KEY = env_vars["GPT_API_KEY"]

df.head()

,item_1_short,Labelling
0,ITEM 1. BUSINESS ﻿ INTRODUCTION ﻿ Evolving Sys...,Evolving Systems provides real-time digital en...
1,"ITEM 1. BUSINESS Our Business We develop, mark...","We develop, market, license, host, and support..."
2,ITEM 1. Business Overview Model N is a leading...,Model N is a leading provider of cloud revenue...
3,Item 1. Business Overview Datadog is the monit...,Datadog is the monitoring and analytics platfo...
4,Item 1. Business Overview AudioEye is an indus...,AudioEye is an industry-leading software solut...


In [11]:
from langchain.llms import OpenAI


def generate_response(input_text):
  llm = OpenAI(temperature=0.7, openai_api_key=GPT_API_KEY)
  return llm(input_text)

print(generate_response("what happens at ING netherlands?"))



At ING Netherlands, customers can access a range of financial services. These services include savings and investments, personal banking, business banking, loans, insurance, and financial planning. ING Netherlands also offers online and mobile banking services, as well as a range of digital products and services.
